In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, Conv2DTranspose
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import backend as K

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96


In [2]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     TRAIN_PATH = os.path.join(base_dir, 'train+val', 'train')
     VALID_PATH = os.path.join(base_dir, 'train+val', 'valid')

     RESCALING_FACTOR = 1./255
     
     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

     train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='input')

     val_gen = datagen.flow_from_directory(VALID_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='input',
                                             shuffle=False)
     
     return train_gen, val_gen

In [3]:
train_gen, val_gen = get_pcam_generators('../')

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [8]:
model = Sequential()

def autoencoder(layers = 1):
    for i in range(layers):
        model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(96,96,3)))
        model.add(MaxPool2D((2,2), padding='same'))

        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(MaxPool2D((2,2), padding='same'))

        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(MaxPool2D((2,2), padding='same'))

        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(UpSampling2D((2,2)))

        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(UpSampling2D((2,2)))
        
        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(UpSampling2D((2,2)))


def get_model():
    autoencoder()
    model.compile(optimizer='adam', loss='binary_crossentropy')

get_model()



In [9]:
train_gen, val_gen = get_pcam_generators('../')

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [10]:
for layer in model.layers:
    print(layer.output_shape)


(None, 96, 96, 64)
(None, 48, 48, 64)
(None, 48, 48, 32)
(None, 24, 24, 32)
(None, 24, 24, 32)
(None, 12, 12, 32)
(None, 12, 12, 32)
(None, 24, 24, 32)
(None, 24, 24, 32)
(None, 48, 48, 32)
(None, 48, 48, 64)
(None, 96, 96, 64)


In [12]:
# save the model and weights
model_name = 'my_first_cnn_model'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 


# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]


# train the model
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

history = model.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=3,
                    shuffle=True,
                    callbacks=callbacks_list)

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node 'binary_crossentropy/mul' defined at (most recent call last):
    File "c:\Users\20212059\.conda\envs\8p361\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\20212059\.conda\envs\8p361\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\20212059\.conda\envs\8p361\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\20212059\.conda\envs\8p361\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\20212059\.conda\envs\8p361\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "c:\Users\20212059\.conda\envs\8p361\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\20212059\.conda\envs\8p361\l